# Data Reading

In [1]:
import pandas as pd
import cv2

In [2]:
data = pd.read_csv( './training_data/driving_log.csv', header=None, names=[
    'center_img_path', 
    'left_img_path',
    'right_img_path',
    'steering_angle',
    'throttle',
    'break',
    'speed'
])

In [3]:
def load_img(src):
    path = str( './training_data/IMG/') +src.split('/')[-1]
    return cv2.imread(path)


data['center_img'] = data.center_img_path.apply(load_img)
data['left_img'] = data.left_img_path.apply(load_img)
data['right_img'] = data.right_img_path.apply(load_img)

In [4]:
data.head(2)

,center_img_path,left_img_path,right_img_path,steering_angle,throttle,break,speed,center_img,left_img,right_img
0,/Users/dennisjost/Desktop/data/IMG/center_2017...,/Users/dennisjost/Desktop/data/IMG/left_2017_0...,/Users/dennisjost/Desktop/data/IMG/right_2017_...,0.0,0.0,0.0,0.000003,"[[[185, 145, 116], [185, 145, 116], [184, 144,...","[[[185, 145, 116], [185, 145, 116], [184, 144,...","[[[185, 145, 116], [185, 145, 116], [184, 144,..."
1,/Users/dennisjost/Desktop/data/IMG/center_2017...,/Users/dennisjost/Desktop/data/IMG/left_2017_0...,/Users/dennisjost/Desktop/data/IMG/right_2017_...,0.0,0.0,0.0,0.000002,"[[[185, 145, 116], [185, 145, 116], [184, 144,...","[[[185, 145, 116], [185, 145, 116], [184, 144,...","[[[185, 145, 116], [185, 145, 116], [184, 144,..."


# Data Generation

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

X = np.asarray( data.center_img.values.tolist() )
y = data[['steering_angle']].values


X_train, X_test, y_train, y_test = train_test_split(X, y)



# Neural Net

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Lambda
from keras.applications import VGG16

#vgg = VGG16( include_top=False, input_shape=(160, 320, 3) )

model = Sequential()
#model.add( vgg )
model.add( Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160, 320, 3) ))
model.add( Flatten() )
model.add( Dense( 1024, activation='relu'))
model.add( Dense(1) )

model.compile( optimizer='adam', loss='mse' )
model.fit( X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=3)



/usr/local/lib/python3.6/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2037 samples, validate on 510 samples
Epoch 1/3
2037/2037 [==============================] - 126s - loss: 190.1658 - val_loss: 0.3486
Epoch 2/3
2037/2037 [==============================] - 124s - loss: 1.8228 - val_loss: 0.4124
Epoch 3/3
2037/2037 [==============================] - 139s - loss: 0.1380 - val_loss: 0.0400


In [10]:
model.save( "2017-07-19-local.h5" )